In [1]:
%matplotlib inline
import pandas as pd
from scipy import stats

In [2]:
parametro = 'TS_1'
xls_data = pd.ExcelFile('data/{}_data.xlsx'.format(parametro))
df_data = xls_data.parse(parametro, index_col='Fecha')

In [3]:
def serieagrupos(sr_entrada):
    df_entrada = pd.DataFrame(sr_entrada)
    df_entrada['Year'] = df_entrada.index.year
    df_entrada['Mes'] = df_entrada.index.month
    gm_salida = df_entrada.pivot(index='Year', columns='Mes')
    gm_salida.columns = gm_salida.columns.droplevel()
    
    return gm_salida


def gruposaserie(gm_entrada):
    sr_entrada = gm_entrada.unstack()
    df_confecha = sr_entrada.reset_index()
    df_confecha['Fecha_str'] = df_confecha['Year'].astype(str) + '-' + df_confecha['Mes'].astype(str) + '-01'
    df_confecha['Fecha'] = pd.to_datetime(df_confecha['Fecha_str'])
    df_confecha.set_index(df_confecha['Fecha'], inplace=True)
    df_confecha.sort_index(inplace=True)

    return df_confecha[0]

In [4]:
def pruebaanomalos(df_input, alpha=0.05, two_tail=True):
    """
    This function applies the Grubbs' Test for outliers in a dataframe and returns two dataframes, the first one
    without outliers and the second one just for the outliers
    :param df_input: Pandas dataframe with series to test.
    :param alpha: Significance level [5% as default].
    :param two_tail: Two tailed distribution [True as default].
    :return: tuple with two dataframes, the first one without outliers and the second one just for outliers.
    """
    df_try = df_input.copy()
    df_output = pd.DataFrame(index=df_input.index, columns=df_input.columns)
    df_outliers = pd.DataFrame(data=0, index=df_input.index, columns=df_input.columns)

    if two_tail:
        alpha /= 2

    while not df_outliers.isnull().values.all():
        mean = df_try.mean()
        std = df_try.std()
        n = len(df_try)
        tcrit = stats.t.ppf(1 - (alpha / (2 * n)), n - 2)
        zcrit = (n - 1) * tcrit / (n * (n - 2 + tcrit ** 2)) ** .5
        df_outliers = df_try.where(((df_try - mean) / std) > zcrit)
        df_output.update(df_input[df_outliers.isnull() == False])
        df_try = df_try[df_outliers.isnull()]

    return df_try, df_output


def calcular_faltantes():
    lista_variables = ['TS_1']  # , 'EV_4', 'HR_1', 'PT_4', 'QL_1', 'TS_1']

    xls_salida = pd.ExcelWriter('results/01_consistencia.xlsx')  # Archivo para guardar resultados

    for variable in lista_variables:
        print('Variable analizada: ' + variable)
        ruta_datos = 'data/' + variable + '_data.xlsx'
        xls_data = pd.ExcelFile(ruta_datos)  # Archivo de los datos de la variable
        df_variable = xls_data.parse(variable, index_col='Fecha')
        estaciones = df_variable.columns
        idx_resultados = pd.Index(estaciones, name='Estacion')
        col_resultados = ['Inicio', 'Fin', 'Longitud', 'Faltantes', 'Faltantes [%]', 'Cumple']
        df_resultados = pd.DataFrame(index=idx_resultados, columns=col_resultados)

        for estacion in estaciones:
            sr_estacion = df_variable[estacion]

            gm_estacion = serieagrupos(sr_estacion)
            gm_limpio, gm_anomalos = pruebaanomalos(gm_estacion)
            sr_limpio = gruposaserie(gm_limpio)

            sr_sin_vacios = sr_limpio.dropna()
            fecha_inicio = sr_sin_vacios.index.min()
            fecha_final = sr_sin_vacios.index.max()
            datos_existentes = len(sr_sin_vacios)
            idx_completo = pd.date_range(start=fecha_inicio, end=fecha_final, freq='MS')
            datos_totales = len(idx_completo)
            faltantes = datos_totales - datos_existentes
            faltantes_por = 100 * float(faltantes) / datos_totales
            cumple = datos_totales > 360 and faltantes_por < 30

            df_resultados.loc[estacion, 'Inicio'] = fecha_inicio
            df_resultados.loc[estacion, 'Fin'] = fecha_final
            df_resultados.loc[estacion, 'Longitud'] = datos_totales
            df_resultados.loc[estacion, 'Faltantes'] = faltantes
            df_resultados.loc[estacion, 'Faltantes [%]'] = faltantes_por
            df_resultados.loc[estacion, 'Cumple'] = cumple
            
            """
            print('\nEstacion: ' + str(estacion))
            print('Incio: ' + str(fecha_inicio))
            print('Fin: ' + str(fecha_final))
            print('Longitud: ' + str(datos_totales))
            print('Faltantes: ' + str(faltantes) + ' (' + str(round(faltantes_por, 2)) + '%)')
            """

        df_resultados.to_excel(xls_salida, variable)

    xls_salida.save()
    
    return df_resultados


In [5]:
consistencia_data = calcular_faltantes()

Variable analizada: TS_1


In [6]:
consistencia_data.head()

,Inicio,Fin,Longitud,Faltantes,Faltantes [%],Cumple
Estacion,,,,,,
11025010,1979-01-01 00:00:00,2016-04-01 00:00:00,448,17,3.79464,True
11035010,1983-04-01 00:00:00,2016-03-01 00:00:00,396,10,2.52525,True
11035020,1983-03-01 00:00:00,2014-06-01 00:00:00,376,19,5.05319,True
11045010,1976-01-01 00:00:00,2016-05-01 00:00:00,485,2,0.412371,True
11075010,1990-08-01 00:00:00,2011-02-01 00:00:00,247,9,3.64372,False


In [7]:
estacion_complementar = 11025010
sr_consistencia_data = consistencia_data.loc[estacion_complementar]
inicio = sr_consistencia_data['Inicio']
fin = sr_consistencia_data['Fin']
sr_data = df_data.loc[inicio:fin, estacion_complementar].copy()
gm_data = serieagrupos(sr_data)
gm_data.head()

Mes,1,2,3,4,5,6,7,8,9,10,11,12
Year,,,,,,,,,,,,
1979,16.3,NaN,16.9,16.9,16.8,16.6,16.8,16.4,16.3,16.3,16.4,16.5
1980,16.6,16.5,17.5,17.2,17.2,17.4,16.9,16.7,16.8,16.4,16.2,16.2
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,16.2,16.6,16.7,16.6,16.7,17.3,16.7,17.2,16.5,16.2,16.6,17.0
1983,18.0,17.9,17.9,17.4,17.8,17.5,17.3,17.2,16.7,16.2,16.8,16.0
1984,15.7,16.1,16.6,16.6,16.3,16.4,16.0,16.3,15.7,16.0,15.7,16.5
1985,NaN,16.5,16.5,16.5,16.8,16.8,16.4,16.0,16.2,15.9,15.8,16.1
1986,16.7,16.0,16.8,16.8,17.0,16.8,17.1,16.7,16.8,16.2,16.6,17.1
1987,17.1,17.8,17.5,17.6,17.4,17.6,17.2,17.2,17.0,16.5,17.2,17.2


In [11]:
gm_complementado = gm_data.interpolate(method='slinear', axis=0, limit_direction='both')
# gm_complementado
sr_complementado = gruposaserie(gm_complementado).loc[inicio:fin]
print(sr_complementado.head())
sr_complementado.interpolate(method='slinear', inplace=True)
print(sr_complementado.head())

Fecha
1979-01-01    16.3
1979-02-01     NaN
1979-03-01    16.9
1979-04-01    16.9
1979-05-01    16.8
Name: 0, dtype: float64
Fecha
1979-01-01    16.300000
1979-02-01    16.615254
1979-03-01    16.900000
1979-04-01    16.900000
1979-05-01    16.800000
Name: 0, dtype: float64
